In [ ]:
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Mounted at /gdrive


In [ ]:
#Drive'daki etiketli dosyayı çekiyorum. Sonuna eklemeye devam edeceğim. Yeni gün process'i gün sonu drive'a kaydedilen csv yi çek. *iş durumuna bağlı
!cp /gdrive/MyDrive/ML-MODELS/NewsTestModels/OpenAI-API/NewsLabeled.csv /content/

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 3.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 26.3 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=9dce9f89932d14249264e8ddbc1dca2bb00ba18b148fd0f6bb51a7e3fc57f0ad
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai


In [ ]:
import openai
import pandas as pd
import csv
import time

In [ ]:
def writeCsv(count,keyword,news,isNegative,label):
   with open("NewsLabeled.csv", "a", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows([(count,keyword,news,isNegative,label)])
 

In [61]:
# OpenAI API anahtarını kullanarak API'yi başlatın
openai.api_key = "YOUR-API-KEY"


# Excel dosyasını oku ve haberleri içeren sütunu seçin
df = pd.read_excel("/gdrive/MyDrive/ML-MODELS/NewsTestModels/OpenAI-API/Data/100kdata-input23.xlsx")

In [62]:
# Haberleri metin olarak ayrıştırın
news_keyword= df["strKeyword"].tolist()
news_text = df["strContent"].tolist()
news_isNegative = df["isNegative"].tolist()
print("Keyword: " + str(len(news_keyword)) + ", Haber Sayısı :" + str(len(news_text)) + ", Haber Etiket Sayısı:" + str(len(news_isNegative)))

Keyword: 191, Haber Sayısı :191, Haber Etiket Sayısı:191


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
counter=0
for i in range(191):

  sentences = news_text[i]
  number_of_sentences = sent_tokenize(sentences)

  if(len(number_of_sentences)<100 and len(sentences)<4096):
    results = openai.Completion.create(
        engine="text-davinci-002",
        #prompt="Haberleri finansal olarak pozitif ya da negatif olarak sınıflandırın: \n\n" + "\n".join(news_text[i]),
        prompt="Haberleri finansal olarak pozitif ya da negatif olarak sınıflandırın: "+ news_text[i] + "###",
        max_tokens=20,
        n=1,
        temperature=0.5,
        stop=["###"]
    )
  
    
    #Sentiment print
    jsonData = results["choices"]
    values=""
    for j in jsonData:
      values = j.get("text")
  
    if(len(values)<100):
      print("---------------------------------------------")
      print(counter)
      print(news_keyword[i])
      print(news_text[i])
      print("Label: "+ values.lower().strip())
      if(values.lower().strip()=="pozitif" or 
         values.lower().strip()=="negatif" or 
         values.lower().strip()=="pozitif." or 
         values.lower().strip()=="negatif."):
        writeCsv(counter,news_keyword[counter],news_text[counter],news_isNegative[counter],(values.capitalize()).strip())
    
    counter+=1
    #promt_size'da patladığı için her değişkende içini boşaltıyorum.
    results=""
    prompt=""
    #Her sorgudan sonra 6 saniye delay
    time.sleep(6)

  

---------------------------------------------
0
ALİ ÇELİK
Kütahya İl İdare Şube Başkanları, Vali Ali Çelik’in başkanlığında bir araya geldi. Vali Ali Çelik, Kurban Bayramı’nda piknik ve mesire alanlarında yoğunluk yaşanabileceğine dikkat çekerek, “Tüm kamu kurumlarımızla orman yangınlarına karşı teyakkuzda olacağız” dedi. Valilik Toplantı Salonunda yapılan toplantıda; yönetim, ekonomik, sağlık, sosyal yardım, kültür, çevre ve şehircilik uygulamalarıyla ilgili... ... 
Label: pozitif
---------------------------------------------
1
İSMAİL KAYA
Yatağan Belediye Başkanı Mustafa Toksöz de imza kampanyasına gelerek imza kampanyasına destek verdi. İmza kampanyasına, Belediye Başkan Yardımcısı İsmail Kaya, Belediye Meclisi Üyeleri, sivil toplum kuruluşları ve siyasi partilerin temsilcileri ve vatandaşlar da destek verdi. ... 
Label: pozitif
---------------------------------------------
2
SONGÜL YILMAZ
Songül Yılmaz BENZER KONULAR 03.02.2022 Artvinspor Kulübü Başkanı İrfan Yıldırım, Rize’ye tayi

In [64]:
!cp /content/NewsLabeled.csv /gdrive/MyDrive/ML-MODELS/NewsTestModels/OpenAI-API/